In [ ]:
import glob
import pandas as pd
import re
import os
import nltk
nltk.download('punkt')
#from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize

INPUT_DIR_PATH = '/Users/salvo/Documents/tesi/datasets/snippets_2000_47_score-2_acc-false_rows-2/'

languages = []
for lang in os.listdir(INPUT_DIR_PATH):
    languages.append(lang)

snippets_dic = {}
for lang in languages:
    files = glob.glob(INPUT_DIR_PATH + lang.lower() + '/*txt')
    print('current lang: ' + lang + ', files:' + str(len(files)))

    cur_snippets = []
    for file in files:
        with open(file, 'r', encoding='utf8') as file:
            cur_snippets.append(file.read())

    snippets_dic[lang] = cur_snippets

snippets = pd.DataFrame(snippets_dic)

In [ ]:
pattern = r'''[\w']+|[""!"#$%&'()*+,-./:;<=>?@\[\]^_`{|}~""]'''

def tokenize(snippet):

    temp = re.sub(r'".*"|\'.*\'', 'strv', snippet)
    return regexp_tokenize(temp, pattern)
    # return word_tokenize(temp)

snippets = snippets.applymap(tokenize)


In [ ]:
snippets.head()

In [ ]:
from gensim.models import Word2Vec

list_data = []
for lang in languages:
    list_data = list_data + snippets[lang].tolist()

w2v_model = Word2Vec(list_data, size=300, min_count=5)

words = list(w2v_model.wv.vocab)
w2v_model.save('../models/w2v_model.bin')

#w2v_model.wv.vocab[word].index

In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec.load('../models/w2v_model.bin')

In [ ]:
find_similar_to = 'print'

print('Similar words to '+find_similar_to+': \n')

# Find similar words, using cosine similarity
# by default shows top 10 similar words
for similar_word in w2v_model.wv.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.3f}".format(
        similar_word[0], similar_word[1]
    ))

In [ ]:
w2v_model.wv['print']